# Organising Data

- These refactoring techniques deal with data handling, letting you replace primitives (with limited functionality) with more appropriate classes

- It also helps to disentangle class associations, which improves portability and reusability

- Problem
    - You have a code fragment that can be grouped

- Solution
    - Group it in a method, and replace the original code with a method call

- Motivation
    - Minimise the number of lines in a method, which makes it easier to figure out what it does

- How to refactor
    - Make a new method and name it self-evidently
    - Copy the code fragment to your new method, and delete the existing fragment, calling the new method instead

- Relationships with other refactoring methods
    - Opposite 
        - Inline Method
    - Similar
        - Move Method
    - Related
        - Introduce Parameter Object
        - Form Template Method
        - Parameterize Method

- Related code smells
    - Duplicate Code
    - Long Method
    - Feature Envy
    - Switch Statements
    - Message Chains
    - Comments
    - Data Class

- Example:

## 1. Self-Encapsulate Field

- Problem
    - You directly access private fields in a class from outside it

- Solution
    - Create a getter and setter, and only uses these for accessing the field.
    - In Python, you can do this by creating "hidden" fields e.g. `self._protected_field`, and using `@property` decorator to define getters and setters to restrict access

- Motivation
    - There are occasions when the usual getting and setting is insufficient, you want to do some logical checks for the field
    - e.g. when setting phone number, probably good to check if the number of digits is correct 

- Drawbacks
    - This logic is useful, but also adds clutter. If you need a getter and setter for every field, then the number of methods you have multiplies by 3

- How to refactor
    - Create a getter (and optional setter) for the field. They should be either protected or public.
    - Find direct invocations and replace with getter/setter methods
    - In Python, idiomatic way is to use @property

- Relationships with other refactoring methods
    - Similar
        - Encapsulate Field
    - Related
        - Duplicate Observed Data
        - Replace Type Codes with Subclasses
        - Replace Type Code with State/Strategy

- Example:

In [ ]:
class bad__Range():
    def __init__(self, low, high):
        self.high = high
        self.low = low

    def check_value(self, value):
        return (value >= self.low) and (value <= self.high)


class good__Range():
    def __init__(self, low, high):
        self._high = high
        self._low = low

    @property
    def high(self):
        return self._high

    @high.setter
    def high(self, value):
        self._high = value

    @property
    def low(self):
        return self._low

    @low.setter
    def low(self, value):
        self._low = value

    def check_value(self, value):
        return (value >= self.low) and (value <= self.high)

## 2. Replace Data Value with Object

- Problem
    - You have a class `A` that has some property `p` stored as a primitive, when ideally `p` can and should have richer data fields and functionality

- Solution
    - Turn `p` into a class and store it as a field in `A`

- Motivation
    - Makes the responsibility of objects clearer
    - Group methods and data into a single relevant class

- How to refactor
    - Create a new class for `p` and pass it into `A`

- Relationships with other refactoring methods
    - Similar
        - Extract Class
        - Introduce Parameter Object
        - Replace Array with Object
        - Replace method with method object

- Related code smells
    - Duplicate Code

- Example:

In [1]:
class bad__Order():
    def __init__(self, customer: str):
        self.customer: str = customer

###

class Customer():
    def __init__(self, name: str):
        self.name: str = name

class good__Order():
    def __init__(self, customer: Customer):
        self.customer: Customer = customer


## 3. Change Value to Reference

- Problem
    - You have many instances of class `A` that contains a reference to class `B`, where every instance of `B` is identical
    - If you instantiate `B` separately for each instance of `A`, you have many copies of `B` floating around in memory. This is inefficient
    - It is also an issue if you need to modify `B` in your code, because you need to modify all instances

- Solution
    - Rather than instantiating identical copies of `B`, for every `A`, instantiate `B` once as a single reference object

- Motivation
    - In most systems, objects are either values or references
        - Reference is where 1 object in your programme corresponds to 1 real world object
        - Values (instances) is where many objects in your programme correspond to the same real world object
            - For example, if I store '2024-01-15' as date1 and '2024-01-15' as date2, the same date is stored in 2 different objects
    - When you have something that changes in your code, and you want it to be consistent across your code, you should store a reference to the same object

- How to refactor
    - Store a place to call an object by some identifier, rather than calling the constructor for each new instance

- Relationships with other refactoring methods
    - Opposite 
        - Change Reference to Value

- Example:

In [3]:
customer_map = {}

class Customer():
    def __init__(self, id: int):
        self.id = id

def create_customer(cid: int):
    ## Lookup map before creating new instance, and return existing intance if it exists
    if cid in customer_map:
        return customer_map.get(cid)
    else:
        customer_map[cid] = Customer(cid)
    return Customer(cid)
customer1 = create_customer(1)
customer1 = create_customer(1)

{1: <__main__.Customer at 0x10620f0d0>}

- In fact, in Python, it's possible to modify the class constructor to achieve this without the need for an external hashmap

In [10]:
class Customer():
    ## Store a map to check if the object with the relevant value has already been created
    _created = {}

    def __new__(cls, name):
        if name in cls._created:
            return cls._created.get(name)
        else:
            ## If object is not yet created, call the __new__ method from the `object` superclass to create a new instance of the class
            cls._created[name] = super().__new__(cls)

    def __init__(self, name):
        self.name = name

ca = Customer('a')
cb = Customer('b')
Customer._created

{'a': <__main__.Customer at 0x1063ccee0>,
 'b': <__main__.Customer at 0x10651d9f0>}

## 4. Change Reference to Value

- Problem
    - Opposite problem to 3. Change Value to Reference
    - An object is too small/infrequently access to justify managing its life cycle

- Solution
    - Just turn it into a value object, instead of a reference object

- Motivation
    - inimise the number of lines in a method, which makes it easier to figure out what it does

- How to refactor
    - Make a new method and name it self-evidently
    - Copy the code fragment to your new method, and delete the existing fragment, calling the new method instead

- Relationships with other refactoring methods
    - Opposite 
        - Inline Method
    - Similar
        - Move Method
    - Related
        - Introduce Parameter Object
        - Form Template Method
        - Parameterize Method

- Related code smells
    - Duplicate Code
    - Long Method
    - Feature Envy
    - Switch Statements
    - Message Chains
    - Comments
    - Data Class

- Example: